
# Multi-omics ALS Prognosis — Detailed Colab Notebook

**Purpose:** Train a multimodal latent dynamical model on synthetic multi-omics ALS data (transcriptomics, proteomics, epigenomics, genomics) I generated previously. The model learns cross-omic relationships and latent dynamics so that at inference time **only one omic (e.g., RNA)** can be provided and the model will predict molecular progression / prognosis.

---

## Project overview (for a bioinformatics audience)

This project explores how **integrating multiple omics layers** (transcriptomics, proteomics, epigenomics, genomics) can improve understanding and prediction of **disease progression in Amyotrophic Lateral Sclerosis (ALS)**.

Instead of focusing on single datasets, the model learns the **molecular relationships between omics layers**, so that even if only one type (e.g. RNA‑seq) is available for a patient, it can infer the likely molecular state across other layers and predict disease progression.

### Biological motivation

ALS is a complex neurodegenerative disorder involving disruptions across multiple molecular levels — gene expression, protein regulation, DNA methylation, and genetic variation. Traditional analyses often treat each omics type separately, which loses cross-layer information. This project aims to model the **shared latent structure** underlying these layers — effectively learning a “molecular fingerprint” of ALS progression.

---

This notebook is **self-contained** and uses the synthetic dataset created earlier in this environment (path: `/mnt/data/synthetic_multiomics_als_realgenes`). It is organized and documented so you can upload it to **Google Colab** and run all cells sequentially.


## Setup — install dependencies and import libraries
This cell installs required packages (uncomment in Colab) and imports commonly used libraries.

In [4]:

# Uncomment the next line when running in Colab
# !pip install --quiet torch numpy pandas scikit-learn matplotlib seaborn joblib

import os, sys, math, random, json, time
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
import joblib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

print("numpy", np.__version__, "pandas", pd.__version__)
print("torch", torch.__version__)


numpy 2.0.2 pandas 2.2.2
torch 2.8.0+cu126


## Load synthetic multi-omics data
We use the synthetic multi-omics dataset generated earlier in this session. Files are expected at `/mnt/data/synthetic_multiomics_als_realgenes`.
If you run in Colab, upload the files to the same path or modify the paths accordingly.

In [5]:

base_path = "/mnt/data/synthetic_multiomics_als_realgenes"
print("base_path exists:", os.path.exists(base_path))

rna_path = os.path.join(base_path, "rna_counts_samples_x_genes.csv")
prot_path = os.path.join(base_path, "proteomics_samples_x_proteins.csv")
epi_path = os.path.join(base_path, "metabolomics_samples_x_metabolites.csv")  # note: used metabolomics earlier as proxy
# In earlier generation methylation file name was 'methylation_samples_x_cpgs.csv' if present, try both
methyl_path = os.path.join(base_path, "methylation_samples_x_cpgs.csv")
geno_path = os.path.join(base_path, "genomics_samples_x_variants.csv")
meta_path = os.path.join(base_path, "sample_metadata.csv")

for p in [rna_path, prot_path, methyl_path, geno_path, meta_path]:
    print(p, os.path.exists(p))

# Load, using metabolomics if methylation isn't present
rna = pd.read_csv(rna_path, index_col=0)
prot = pd.read_csv(prot_path, index_col=0)
if os.path.exists(methyl_path):
    epi = pd.read_csv(methyl_path, index_col=0)
else:
    # fallback to metabolomics file if methylation not found
    epi = pd.read_csv(epi_path, index_col=0)
geno = pd.read_csv(geno_path, index_col=0)
meta = pd.read_csv(meta_path)

# Align samples (order)
samples = list(rna.index)
prot = prot.reindex(samples)
epi = epi.reindex(samples)
geno = geno.reindex(samples)
meta = meta.set_index("sample_id").reindex(samples)

print("Samples count:", len(samples))
print("RNA shape:", rna.shape, "Proteomics shape:", prot.shape, "Epi shape:", epi.shape, "Geno shape:", geno.shape)
meta.head()


base_path exists: False
/mnt/data/synthetic_multiomics_als_realgenes/rna_counts_samples_x_genes.csv False
/mnt/data/synthetic_multiomics_als_realgenes/proteomics_samples_x_proteins.csv False
/mnt/data/synthetic_multiomics_als_realgenes/methylation_samples_x_cpgs.csv False
/mnt/data/synthetic_multiomics_als_realgenes/genomics_samples_x_variants.csv False
/mnt/data/synthetic_multiomics_als_realgenes/sample_metadata.csv False


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/synthetic_multiomics_als_realgenes/rna_counts_samples_x_genes.csv'

## Quick exploratory checks

In [ ]:

print("Groups distribution:\n", meta['group'].value_counts())
print("\nRNA example genes:", list(rna.columns[:8]))
display(rna.iloc[:5, :8])
display(prot.iloc[:5, :8])
display(epi.iloc[:5, :8])
display(geno.iloc[:5, :8])


## Preprocessing
Normalize each omic appropriately, select top variable features to keep the model compact.

In [6]:

# RNA: log2 CPM
def normalize_log2_cpm(df):
    mat = df.values.astype(float)
    lib = mat.sum(axis=1, keepdims=True)
    cpm = (mat / (lib + 1e-9)) * 1e6
    return pd.DataFrame(np.log2(cpm + 1.0), index=df.index, columns=df.columns)

rna_log2 = normalize_log2_cpm(rna)
prot_log = np.log1p(prot)  # proteomics log scale
# epi: if methylation (0-1) convert to M-values; if metabolomics keep log
if ((epi.values >= 0).all() and (epi.values <= 1).all()):
    # treat as methylation beta -> M-values
    epi_m = pd.DataFrame(np.log((epi+1e-6) / (1 - epi + 1e-6)), index=epi.index, columns=epi.columns)
else:
    epi_m = np.log1p(epi)

geno_bin = geno.copy()  # assume 0/1

# Feature selection: keep top variance features per modality
def select_top_var(df, k):
    if df.shape[1] <= k:
        return df
    vars_ = df.var(axis=0)
    keep = vars_.sort_values(ascending=False).index[:k]
    return df[keep]

rna_sel = select_top_var(rna_log2, 800)
prot_sel = select_top_var(prot_log, 150)
epi_sel = select_top_var(epi_m, 150)
geno_sel = geno_bin  # keep all variants (likely small)

print("Selected shapes:", rna_sel.shape, prot_sel.shape, epi_sel.shape, geno_sel.shape)


NameError: name 'rna' is not defined

## Standardize features and prepare baseline/follow-up
We will simulate a follow-up (6 months) by applying a small drift correlated with an artificial progression slope per sample — this mirrors how the previous prototype created longitudinal pairs.

In [7]:

from sklearn.preprocessing import StandardScaler

# Fit scalers per modality
scaler_rna = StandardScaler().fit(rna_sel.values)
scaler_prot = StandardScaler().fit(prot_sel.values)
scaler_epi = StandardScaler().fit(epi_sel.values)
scaler_geno = StandardScaler().fit(geno_sel.values.astype(float))

X_rna = scaler_rna.transform(rna_sel.values)
X_prot = scaler_prot.transform(prot_sel.values)
X_epi = scaler_epi.transform(epi_sel.values)
X_geno = scaler_geno.transform(geno_sel.values.astype(float))

N = X_rna.shape[0]
groups = list(meta['group'].values)

# create synthetic progression slopes (ALS negative)
slopes = np.array([ -np.random.uniform(0.2,1.2) if g=='ALS' else np.random.normal(0,0.05) for g in groups ])

# function to make followup with small driver changes correlated to slope
def make_followup(X, slope, frac_drivers=0.05):
    X1 = X.copy()
    n_feats = X.shape[1]
    n_driver = max(1, int(n_feats * frac_drivers))
    for i in range(X.shape[0]):
        drivers = np.random.choice(n_feats, size=n_driver, replace=False)
        change = (np.sign(-slope[i]) * 0.05 * abs(slope[i]))
        X1[i, drivers] += change + np.random.normal(0,0.02, size=n_driver)
        X1[i] += np.random.normal(0, 0.01, size=n_feats)
    return X1

Xr0 = X_rna.copy(); Xr1 = make_followup(Xr0, slopes, frac_drivers=0.03)
Xp0 = X_prot.copy(); Xp1 = make_followup(Xp0, slopes, frac_drivers=0.05)
Xe0 = X_epi.copy(); Xe1 = make_followup(Xe0, slopes, frac_drivers=0.04)
Xg0 = X_geno.copy(); Xg1 = Xg0.copy()  # variants static

print("Prepared baseline and followup arrays: N=", N)


NameError: name 'rna_sel' is not defined

## Model architecture
We implement modality-specific encoders and decoders (MLPs), combine posteriors with a Product-of-Experts (PoE) to get a joint `z`, learn linear latent dynamics `z_{t+1} = A z_t + b`, and train a small prognostic head on top of `[z0, dz]`.

Details and hyperparameters are commented in the code.

In [8]:

# Model components
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)

class MLPEncoder(nn.Module):
    def __init__(self, in_dim, hidden=[512,256], latent=32):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden[0])
        self.fc2 = nn.Linear(hidden[0], hidden[1])
        self.mu = nn.Linear(hidden[1], latent)
        self.logv = nn.Linear(hidden[1], latent)
        self.act = nn.ReLU()
    def forward(self, x):
        h = self.act(self.fc1(x))
        h = self.act(self.fc2(h))
        return self.mu(h), self.logv(h)

class MLPDecoder(nn.Module):
    def __init__(self, out_dim, hidden=[256,512], latent=32):
        super().__init__()
        self.d1 = nn.Linear(latent, hidden[0])
        self.d2 = nn.Linear(hidden[0], hidden[1])
        self.out = nn.Linear(hidden[1], out_dim)
        self.act = nn.ReLU()
    def forward(self, z):
        h = self.act(self.d1(z))
        h = self.act(self.d2(h))
        return self.out(h)

def poe(mu_list, logvar_list, eps=1e-6):
    # Product-of-Experts for Gaussian posteriors
    var_list = [torch.exp(lv) + eps for lv in logvar_list]
    inv_var = [1.0 / v for v in var_list]
    mu = sum(m*iv for m,iv in zip(mu_list, inv_var)) / sum(inv_var)
    var = 1.0 / sum(inv_var)
    logvar = torch.log(var + eps)
    return mu, logvar

class MultiModalVAE(nn.Module):
    def __init__(self, dims, latent=32):
        super().__init__()
        # dims: dict with input dims per modality
        self.enc_rna = MLPEncoder(dims['rna'], latent=latent)
        self.enc_prot = MLPEncoder(dims['prot'], latent=latent)
        self.enc_epi = MLPEncoder(dims['epi'], latent=latent)
        self.enc_geno = MLPEncoder(dims['geno'], latent=latent)

        self.dec_rna = MLPDecoder(dims['rna'], latent=latent)
        self.dec_prot = MLPDecoder(dims['prot'], latent=latent)
        self.dec_epi = MLPDecoder(dims['epi'], latent=latent)
        self.dec_geno = MLPDecoder(dims['geno'], latent=latent)

        # linear dynamics layer (learnable A via Linear)
        self.dyn = nn.Linear(latent, latent, bias=True)

        # prognostic head: maps [z0, z1_pred - z0] -> scalar slope
        self.prog = nn.Sequential(nn.Linear(latent*2, latent), nn.ReLU(), nn.Linear(latent,1))

        self.latent = latent

    def encode_modal(self, x, modality):
        if modality=='rna': return self.enc_rna(x)
        if modality=='prot': return self.enc_prot(x)
        if modality=='epi': return self.enc_epi(x)
        if modality=='geno': return self.enc_geno(x)
        raise ValueError('Unknown modality')

    def decode_modal(self, z, modality):
        if modality=='rna': return self.dec_rna(z)
        if modality=='prot': return self.dec_prot(z)
        if modality=='epi': return self.dec_epi(z)
        if modality=='geno': return self.dec_geno(z)
        raise ValueError('Unknown modality')

    def sample_z(self, mu, logv):
        std = torch.exp(0.5*logv)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, inputs):
        # inputs: dict of tensors or None for each modality
        mus=[]; logvs=[]
        for m in ['rna','prot','epi','geno']:
            x = inputs.get(m, None)
            if x is not None:
                mu, logv = self.encode_modal(x, m)
                mus.append(mu); logvs.append(logv)
        # joint posterior via PoE
        mu_joint, logvar_joint = poe(mus, logvs)
        z = self.sample_z(mu_joint, logvar_joint)
        # reconstructions for all modalities from joint z
        recons = {m: self.decode_modal(z, m) for m in ['rna','prot','epi','geno']}
        return recons, mu_joint, logvar_joint, z

    def predict_next_z(self, z):
        return self.dyn(z)

    def prognostic(self, z0, z1_pred):
        x = torch.cat([z0, z1_pred - z0], dim=1)
        return self.prog(x).squeeze(1)

# instantiate model
dims = {'rna': Xr0.shape[1], 'prot': Xp0.shape[1], 'epi': Xe0.shape[1], 'geno': Xg0.shape[1]}
latent_dim = 32
model = MultiModalVAE(dims, latent=latent_dim).to(device)
print('Model initialized with latent dim', latent_dim)


device: cpu


NameError: name 'Xr0' is not defined

## Prepare tensors and helper functions
We prepare PyTorch tensors and a small helper to sample minibatches with modality dropout (to teach encoders robustness to missing data).

In [ ]:

# Prepare tensors
tensor_r0 = torch.tensor(Xr0, dtype=torch.float32).to(device)
tensor_p0 = torch.tensor(Xp0, dtype=torch.float32).to(device)
tensor_e0 = torch.tensor(Xe0, dtype=torch.float32).to(device)
tensor_g0 = torch.tensor(Xg0, dtype=torch.float32).to(device)

tensor_r1 = torch.tensor(Xr1, dtype=torch.float32).to(device)
tensor_p1 = torch.tensor(Xp1, dtype=torch.float32).to(device)
tensor_e1 = torch.tensor(Xe1, dtype=torch.float32).to(device)
tensor_g1 = torch.tensor(Xg1, dtype=torch.float32).to(device)

slopes_t = torch.tensor(slopes, dtype=torch.float32).to(device)

N = Xr0.shape[0]
batch_size = 32

def sample_batch(idx_batch, drop_prob=0.25):
    inputs0 = {}
    for name, t in [('rna', tensor_r0), ('prot', tensor_p0), ('epi', tensor_e0), ('geno', tensor_g0)]:
        if np.random.rand() > drop_prob:
            inputs0[name] = t[idx_batch]
        else:
            inputs0[name] = None
    inputs1 = {'rna': tensor_r1[idx_batch], 'prot': tensor_p1[idx_batch], 'epi': tensor_e1[idx_batch], 'geno': tensor_g1[idx_batch]}
    return inputs0, inputs1, slopes_t[idx_batch]

print('Tensors ready. N=', N, 'batch_size=', batch_size)


## Pretraining: VAE with cross-reconstruction and latent dynamics
We train the multimodal VAE with modality dropout and include a dynamics loss so the latent captures temporal change.

In [9]:

# Training hyperparameters
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
n_epochs = 100
beta = 0.1
lambda_cross = 1.0
gamma_dyn = 1.0

model.train()
for epoch in range(n_epochs):
    perm = np.random.permutation(N)
    epoch_loss = 0.0
    for i in range(0, N, batch_size):
        batch_idx = perm[i:i+batch_size]
        inputs0, inputs1, slopes_b = sample_batch(batch_idx, drop_prob=0.25)
        recons0, mu0, logv0, z0 = model(inputs0)
        # reconstruction loss for modalities present at time0
        recon_loss = 0.0
        for m in ['rna','prot','epi','geno']:
            if inputs0.get(m) is not None:
                recon_loss += F.mse_loss(recons0[m], inputs0[m])
        # dynamics: get true z1 by encoding full modalities at time1 (no dropout)
        mus1=[]; logvs1=[]
        for m in ['rna','prot','epi','geno']:
            x1 = inputs1[m]
            mu1, logv1 = model.encode_modal(x1, m)
            mus1.append(mu1); logvs1.append(logv1)
        mu1_joint, logv1_joint = poe(mus1, logvs1)
        z1_true = model.sample_z(mu1_joint, logv1_joint).detach()
        z1_pred = model.predict_next_z(z0)
        dyn_loss = F.mse_loss(z1_pred, z1_true)
        # KL
        kl = -0.5 * torch.mean(1 + logv0 - mu0.pow(2) - logv0.exp())
        loss = recon_loss + lambda_cross * recon_loss + beta * kl + gamma_dyn * dyn_loss
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        epoch_loss += loss.item()
    if (epoch+1) % 10 == 0 or epoch==0:
        print(f"Epoch {epoch+1}/{n_epochs} loss {epoch_loss/N:.6f}")


NameError: name 'model' is not defined

## Fit linear latent dynamics (Ridge) using encoded means
After pretraining, encode every sample's baseline and follow-up using full modalities to obtain `Z0` and `Z1`. Fit a Ridge regression mapping `Z0 -> Z1` as a robust dynamics estimator.

In [10]:

model.eval()
with torch.no_grad():
    mus0=[]; logvs0=[]
    mus1=[]; logvs1=[]
    for m, t0, t1 in [('rna', tensor_r0, tensor_r1), ('prot', tensor_p0, tensor_p1), ('epi', tensor_e0, tensor_e1), ('geno', tensor_g0, tensor_g1)]:
        mu0_m, logv0_m = model.encode_modal(t0, m)
        mu1_m, logv1_m = model.encode_modal(t1, m)
        mus0.append(mu0_m); logvs0.append(logv0_m)
        mus1.append(mu1_m); logvs1.append(logv1_m)
    mu0_joint, logv0_joint = poe(mus0, logvs0)
    mu1_joint, logv1_joint = poe(mus1, logvs1)
    Z0 = mu0_joint.cpu().numpy()
    Z1 = mu1_joint.cpu().numpy()

from sklearn.linear_model import Ridge
ridge_dyn = Ridge(alpha=1.0).fit(Z0, Z1)
Z1_pred = ridge_dyn.predict(Z0)
print("Latent dynamics MSE:", mean_squared_error(Z1, Z1_pred))


NameError: name 'model' is not defined

## Train prognostic head and evaluate RNA-only inference
We train a simple Ridge regression mapping latent features & predicted latent slope to the clinical progression slope. Then evaluate the model when only RNA is provided at inference.

In [ ]:

# Train/test split (patient-level)
from sklearn.model_selection import train_test_split
idx = np.arange(N)
train_idx, test_idx = train_test_split(idx, test_size=0.2, random_state=42, stratify=np.array(groups))

# Compute RNA-only encoded z0 (means)
model.eval()
with torch.no_grad():
    mu_rna0, logv_rna0 = model.encode_modal(torch.tensor(Xr0, dtype=torch.float32).to(device), 'rna')
    Z0_rna = mu_rna0.cpu().numpy()

# z1 from ridge dyn applied to RNA-only z0
Z1_from_rna = ridge_dyn.predict(Z0_rna)
Xprog_all = np.hstack([Z0, (Z1 - Z0)])  # full-modal training features
# train ridge prognostic on full-modal latent features (simulate availability in training)
ridge_prog = Ridge(alpha=1.0).fit(Xprog_all, slopes)

# Evaluate RNA-only inference on test set
Xprog_rna = np.hstack([Z0_rna, (Z1_from_rna - Z0_rna)])
y_pred = ridge_prog.predict(Xprog_rna[test_idx])
y_true = slopes[test_idx]
print("RNA-only prognostic R2:", r2_score(y_true, y_pred))
print("MSE:", mean_squared_error(y_true, y_pred))


## Visualizations & save artifacts
PCA of joint latent `Z0` and scatter of true vs predicted progression (RNA-only). Save model and scalers.

In [11]:

from sklearn.decomposition import PCA
pca = PCA(2).fit(Z0)
Z0_2 = pca.transform(Z0)
plt.figure(figsize=(6,5))
for g in np.unique(groups):
    mask = np.array(groups) == g
    plt.scatter(Z0_2[mask,0], Z0_2[mask,1], label=g, alpha=0.7)
plt.legend(); plt.title('Joint latent Z0 PCA'); plt.show()

# RNA-only pred vs true
plt.figure(figsize=(6,4))
plt.scatter(y_true, y_pred, alpha=0.7)
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--')
plt.xlabel('True slope'); plt.ylabel('Predicted slope'); plt.title('RNA-only prognostic'); plt.show()

# Save model + scalers + ridge models
out_dir = "/mnt/data/multiomics_results"
os.makedirs(out_dir, exist_ok=True)
torch.save(model.state_dict(), os.path.join(out_dir, "vae_state.pth"))
joblib.dump({'scaler_rna': scaler_rna, 'scaler_prot': scaler_prot, 'scaler_epi': scaler_epi, 'scaler_geno': scaler_geno}, os.path.join(out_dir, "scalers.pkl"))
joblib.dump({'ridge_dyn': ridge_dyn, 'ridge_prog': ridge_prog}, os.path.join(out_dir, "models.pkl"))
print("Saved artifacts to", out_dir)


NameError: name 'Z0' is not defined


## Conclusion and next steps

This notebook demonstrated a full pipeline:
- Load synthetic multi-omics ALS data (real gene names).
- Preprocess and create baseline/follow-up pairs.
- Train a multimodal VAE with cross-reconstruction and latent dynamics.
- Fit a robust linear dynamics model in latent space.
- Train a prognostic head and evaluate **RNA-only** inference.

**Next steps** you may want to run:
- Replace synthetic labels with real longitudinal ALSFRS-R or survival data.
- Increase sample size and include batch effects for realism.
- Replace linear dynamics with latent ODE for irregular sampling.
- Add explainability (SHAP) on the prognostic head and pathway enrichment for latent dimensions.

If you want, I can now run this entire notebook here and show the outputs (plots, metrics), or package the notebook as a `.ipynb` file for you to download and run in Colab.
